# Selenium Scraper für Geschäfte im Kantonsrat
Dieses Script lädt alle Geschäfte des Kantonsrats herunter. Name/Nummer/Beteiligte.

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from cycler import cycler
plt.rcParams.update(
    {"figure.facecolor": "#ffffff",
              "axes.facecolor": "#ffffff",
              "axes.grid" : True,
              "axes.grid.axis" : "y",
              "grid.color"    : "ededed",
              "grid.linewidth": 0.8,
              "grid.alpha": 0.8,
              "axes.spines.left" : False,
              "axes.spines.right" : False,
              "axes.spines.top" : False,
              "axes.spines.bottom" : False,
              "axes.axisbelow": False,
              "ytick.major.size": 0,     
              "ytick.minor.size": 0,
              #"xtick.direction" : "none",
              "xtick.minor.size": 0,
              "xtick.major.size" : 0,
              "xtick.color"      : "#191919",
              "axes.edgecolor"    :"#191919",
              "axes.prop_cycle" : plt.cycler('color', ['#0c2c84', '#c7e9b4', 
                                                       '#225ea8','#1d91c0', '#41b6c4', 
                                                       '#7fcdbb', '#eaea8c']),
    'pdf.fonttype': 42,
    'ps.fonttype': 42
})
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
import re
import time


In [ ]:
# Open Chrome Webdriver for Selenium
driver = webdriver.Chrome()

In [ ]:
# send webdriver to webstie
driver.get('https://www.kantonsrat.zh.ch/geschaefte/geschaefte.aspx')

In [ ]:
#click black searchfield once
button = driver.find_element_by_xpath('//*[@id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_btnSearchGeschaefte"]') #identify search field
driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search
button.click()

In [ ]:
#check how many a-tags are there. Relevant to see if it is last page (51 = not last page)
links = driver.find_elements_by_tag_name('a')
print(len(links))

In [ ]:
#check for the names | len is becoming important for the while loop. if len is bigger than 10, search went wrong. retry
names = driver.find_elements_by_tag_name('tr')
print(names[-9].text)
len(names)

### Lade die Liste aller Geschäftsnummern und Geschäftsnamen herunter

In [ ]:
#save search results for first page
geschaeftsnr = []
geschaeftsname = []
for link in links[17:-9]:
    nr = link.text
    geschaeftsnr.append(nr)
for name in names[3:-8]:
    name = name.text
    geschaeftsname.append(name)

In [ ]:
len(geschaeftsname)

In [ ]:
number_sites = int(re.search('\d\d\d\d*',driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_gridGeschaefte_ctl00"]/tfoot/tr/td/table/tbody/tr/td/div[2]').text)[0])
print(number_sites)

In [ ]:
numbers = list(range(2,number_sites+1)) #numbers to start the loop for the second page

In [ ]:
#start the loop for alle pages | save number and name
for site in numbers:
    print(len(geschaeftsnr))
    print(site)
    button = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_gridGeschaefte_ctl00"]/tfoot/tr/td/table/tbody/tr/td/div[1]/input[3]') #identify search field
    driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search
    button.click()
    time.sleep(1)
    links = driver.find_elements_by_tag_name('a')
    names = driver.find_elements_by_tag_name('tr')
    for link in links[17:-9]:
        nr = link.text
        geschaeftsnr.append(nr)
    for name in names[3:-8]:
        name = name.text
        geschaeftsname.append(name)

In [ ]:
#safe list of dictionaries to csv
#df_nr_name = pd.DataFrame({'geschaeftnr':geschaeftsnr,
#                          'geschaftname':geschaeftsname})
#df_nr_name.to_csv('nummern_name.csv', index=False)

### Hole die Detail-Informationen der Geschäfte

In [ ]:
df_nr_name = pd.read_csv('nummern_name.csv')
print(len(df_nr_name))


In [ ]:
#Use regex to make sepearate columns for numbers and years
df_nr_name['NR'] = df_nr_name['geschaeftnr'].str.extract('(\d*)/')
df_nr_name['NR'] =df_nr_name.NR.astype(int)
df_nr_name['YEAR'] = df_nr_name['geschaeftnr'].str.extract('/(\d*)')
df_nr_name['YEAR'] =df_nr_name.YEAR.astype(int)
df_nr_name.head()

Zuerst: Einzelner Suchdurchlauf, um zu sehen ob alles funktioniert, bevor der Loop gebaut wird

In [ ]:
# send driver to website
driver.get('https://www.kantonsrat.zh.ch/geschaefte/geschaefte.aspx')

In [ ]:
#select field for year, sende value
yearselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_cbxJahr_Input"]')
yearselector.send_keys('2014')

In [ ]:
#select field for policy, sende value
geschaeftselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_tbxLaufNr"]')
geschaeftselector.send_keys('293')
geschaeftselector.send_keys(Keys.RETURN)

In [ ]:
#click on search result
links = driver.find_elements_by_tag_name('a')
links[17].click()

In [ ]:
driver.find_elements_by_tag_name('p')[0].text #geschäftsart

In [ ]:
driver.find_elements_by_tag_name('p')[1].text #beteiligte personen

In [ ]:
driver.find_elements_by_tag_name('h4')[0].text #ablaufschritte

Vorbereitung des Loops

In [ ]:
len(driver.find_elements_by_tag_name('tr'))>10 #if condition für while-loop. if searchresult is not unique len is bigger than 10

In [ ]:
#erease "vorlagen" from the government as we are not interested in those issues. they are always numbere 1000+
ohnevorlagen = df_nr_name[(df_nr_name['NR']<1000)]
ohnevorlagen.head()

In [ ]:
#added new policy, released after first download
neue_geschaefte = ['377/2018','378/2018','379/2018','380/2018','381/2018','382/2018','383/2018','384/2018','385/2018','386/2018','387/2018','388/2018','389/2018']

In [ ]:
#generating list for loop with number
geschaeftsnr = ohnevorlagen['geschaeftnr']
geschaeftsnr = neue_geschaefte + list(geschaeftsnr)

Starte den Loop

In [ ]:
# If you want to open Chrome
driver = webdriver.Chrome()

In [ ]:
data = []
number=0
for number in geschaeftsnr:
    dic = {'nr': 0,
       'year':0,
        'geschaeft': 0,
        'beteiligte':0,
        #'ablaufschritte':0,
          'name':0}
    driver.get('https://www.kantonsrat.zh.ch/geschaefte/geschaefte.aspx')
    gnr = re.search('(\d*)',str(number))[0]
    year = re.search('/(\d\d\d\d)',str(number))[0].replace("/","")
    yearselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_cbxJahr_Input"]')
    yearselector.send_keys(year)
    
    geschaeftselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_tbxLaufNr"]')
    geschaeftselector.send_keys(gnr)
    geschaeftselector.send_keys(Keys.RETURN)
    
    while len(driver.find_elements_by_tag_name('tr'))>11: #weil manchmal Suche nicht eindeutig
            print("_-_-_-_")
            print('oops too many search results, rewind')
            print("_-_-_-_")

            driver.get('https://www.kantonsrat.zh.ch/geschaefte/geschaefte.aspx')
            gnr = re.search('(\d*)',str(number))[0]
            year = re.search('/(\d\d\d\d)',str(number))[0].replace("/","").strip()
            #print(year)
            yearselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_cbxJahr_Input"]')
            yearselector.send_keys(year)
    
            geschaeftselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_8_tbxLaufNr"]')
            geschaeftselector.send_keys(gnr)
            geschaeftselector.send_keys(Keys.RETURN)

        

    links = driver.find_elements_by_tag_name('a')
    links[17].click()
    
    dic['nr'] = gnr
    dic['year'] = year
    dic['geschaeft'] = driver.find_elements_by_tag_name('p')[0].text #geschäftsart
    dic['beteiligte'] = driver.find_elements_by_tag_name('p')[1].text #beteiligte personen
    dic['name'] = driver.find_elements_by_tag_name('h4')[0].text
    #if 'Verknüpfte Geschäfte' in driver.find_elements_by_tag_name('h4')[2].text:
    #    dic['ablaufschritte'] = driver.find_elements_by_tag_name('p')[3].text #ablaufschritte
    #else:
    #    dic['ablaufschritte'] = driver.find_elements_by_tag_name('p')[2].text #ablaufschritte
    print("*****")
    print(dic['nr'], dic['year'])
    print(dic['name'])
    print(dic['beteiligte'])
    data.append(dic)
    

In [ ]:
#Save downloaded policy to csv
#df_dic = pd.DataFrame(data)
#df.to_csv('vorstoesse_2018_12_17.csv', index=False)